In [ ]:
###Feature Selection####
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df= pd.read_csv('labeled_Data.csv')
features=df.columns[1:]
x=df["features"]
y=df["label"]

##Get correlations of each feature in a data set
corrmat=df.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
##Plot heatmap##
g=sns.heatmap(df[top_corr_features].corr(),annot=True,cmap="RdYlGn")


In [ ]:
###Colinerity calculation###
from sklearn.linear_model import LinearRegression
def calculate_vif(df,features):
    vif,tolerance= {},{}
    for feature in features:
        X=[f for f in features if f != feature]
        X, y= df[X], df[feature]
        #extract r-squared from teh fit
        r2= LinearRegression().fit(X,y).score(X,y)
        #Calculate tolerance
        tolerance[feature]=1 - r2
        #Calculate VIF
        vif[feature]=1/(tolerance[feature])
    return pd.DataFrame({'VIF':vif, 'Tolerance':tolerance})

df_vif=calculate_vif(df,features)

##Remove the features with VIF > 5####
df_vif=df_vif[df_vif["VIF"]< 5]
features= df_vif.index.values
df_vif=calculate_vif(df,features)

In [ ]:
###Training the model 


from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X=df[feature]
Y=df["label"]

X= preprocessing.StandardScaler().fit(X).transform(X)
X_train, X_test, Y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=0)
clf_svm= SVC(C=25, kernel="rbf", gamma=0.01, probability= True)
clf_svm.fit(X_train,Y_train)

###Testing the Model
y_pred_svm =clf_svm.predict(X_test)
accuracy= accuracy_score(y_test,y_pred_svm)

##Hyper parameter Tuning####

param_grid = {
    'C': [0.1, 1, 10, 100],           # Regularization parameter
    'kernel': ['linear', 'rbf', 'poly'],  # Kernel type
    'gamma': ['scale', 'auto'],         # Kernel coefficient for ‘rbf’, ‘poly’ kernels
    'degree': [3, 4, 5]                # Degree of the polynomial kernel (if 'poly' is chosen)
}

# Initialize GridSearchCV with the SVC model and the parameter grid
grid_search = GridSearchCV(svc, param_grid, cv=5, scoring='accuracy')

# Fit the model with training data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

# Use the best estimator for predictions
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# You can also evaluate the performance on the test set
from sklearn.metrics import accuracy_score
print("Test accuracy: ", accuracy_score(y_test, y_pred))

Best parameters found:  {'C': 1, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
Best cross-validation accuracy:  0.9714285714285715
Test accuracy:  0.9777777777777777
